In [1]:
# Import Modules
import pandas as pd
import os
import json
import requests
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
# Load .env environment variables into the notebook
env_path = Path("rapidapi.env")
load_dotenv(env_path)

True